In [1]:
import numpy as np
import pandas as pd

### Helper Functions

In [ ]:
def data_quality_report_latex(df, caption="Data Quality Report", label="tab:data_quality"):
    """
    Generate a LaTeX table summarizing data quality for continuous (numeric) features in a DataFrame.
    """
    # Try to convert all columns to numeric, non-convertible entries become NaN
    df_numeric = df.apply(pd.to_numeric, errors='coerce')
    numeric_df = df_numeric.select_dtypes(include=[np.number])

    if numeric_df.empty:
        return "% No numeric columns found in DataFrame.\n"

    report = pd.DataFrame({
        "Feature": numeric_df.columns,
        "Count": numeric_df.shape[0],
        "% Miss.": numeric_df.isnull().sum().values / numeric_df.shape[0] * 100,
        "Card.": numeric_df.nunique().values,
        "Min.": numeric_df.min().values,
        "1st Qrt.": numeric_df.quantile(0.25).values,
        "Mean": numeric_df.mean().values,
        "Median": numeric_df.median().values,
        "3rd Qrt.": numeric_df.quantile(0.75).values,
        "Max.": numeric_df.max().values,
        "Std. Dev.": numeric_df.std().values
    })

    # Format as LaTeX
    latex = report.to_latex(
        index=False,
        float_format="%.2f",
        column_format="|l|" + "r|" * (len(report.columns) - 1),
        caption=caption,
        label=label
    )
    
    return latex


In [3]:
def categorical_data_quality_report_latex(df, caption="Categorical Data Quality Report", label="tab:categorical_quality"):
    """
    Generate a LaTeX table summarizing data quality for categorical features in a DataFrame.
    Treats '?' as a missing value.
    """
    # Replace '?' with np.nan
    df = df.replace('?', np.nan)

    # Select only object and category columns
    cat_df = df.select_dtypes(include=["object", "category"])

    if cat_df.empty:
        return "% No categorical columns found in DataFrame.\n"

    data = []

    for col in cat_df.columns:
        series = cat_df[col]
        total = len(series)
        count = df.shape[0]
        pct_miss = ((total - count) / total * 100) if total > 0 else 0
        cardinality = series.nunique(dropna=True)

        value_counts = series.value_counts(dropna=True)

        mode = value_counts.index[0] if not value_counts.empty else ""
        mode_freq = value_counts.iloc[0] if not value_counts.empty else 0
        mode_pct = (mode_freq / count * 100) if count > 0 else 0

        if len(value_counts) > 1:
            second_mode = value_counts.index[1]
            second_mode_freq = value_counts.iloc[1]
            second_mode_pct = (second_mode_freq / count * 100) if count > 0 else 0
        else:
            second_mode = ""
            second_mode_freq = 0
            second_mode_pct = 0

        data.append([
            col, count, pct_miss, cardinality,
            mode, mode_freq, mode_pct,
            second_mode, second_mode_freq, second_mode_pct
        ])

    report = pd.DataFrame(data, columns=[
        "Feature", "count", "%miss", "Card",
        "Mode", "Mode Freq.", "Mode %",
        "2nd Mode", "2nd Mode Freq", "2nd Mode %"
    ])

    # Format LaTeX table
    latex = report.to_latex(
        index=False,
        float_format="%.2f",
        column_format="|l|r|r|r|l|r|r|l|r|r|",
        caption=caption,
        label=label,
        escape=True
    )

    return latex

# Task 1

In [4]:
df = pd.read_csv("badDataSet.csv", delimiter='\t', header=None, na_values='?')

In [5]:
df_noTarget = df.iloc[:, :-1]

In [6]:
df

,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,62
0,4264440.30,89.0,222.711631,11.0,785.0,81.916660,-0.613049,4352,237.0,222,...,0,0,0,0,0,0,1,1,1,7.0
1,4264440.30,4.0,10.035728,15.0,849.0,81.973876,-0.791090,4413,199.0,209,...,0,0,0,0,0,0,1,2,1,7.0
2,4263334.95,342.0,855.348205,16.0,870.0,81.035217,0.519791,4443,186.0,214,...,0,0,0,0,0,0,1,3,1,7.0
3,4262229.60,24.0,60.238933,15.0,828.0,80.054099,0.432647,4383,211.0,208,...,0,0,0,0,0,0,1,4,1,7.0
4,4258913.55,338.0,847.158476,15.0,892.0,77.054464,-0.923799,4473,187.0,218,...,0,0,0,0,0,0,1,5,1,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2062583.10,23.0,57.594866,14.0,85.0,16.059535,-0.103136,108,212.0,210,...,0,0,0,0,0,0,0,581008,1,3.0
581008,2059267.05,37.0,92.585707,17.0,120.0,17.977536,0.417243,90,217.0,202,...,0,0,0,0,0,0,0,581009,1,6.0
581009,2057056.35,35.0,87.547136,14.0,60.0,10.962117,0.902718,85,218.0,209,...,0,0,0,0,0,0,0,581010,1,3.0
581010,2055951.00,18.0,45.141729,13.0,95.0,14.972901,-0.202035,90,210.0,213,...,0,0,0,0,0,0,0,581011,1,3.0


In [7]:
df_noTarget

,0,1,2,3,4,5,6,7,8,9,...,52,53,54,55,56,57,58,59,60,61
0,4264440.30,89.0,222.711631,11.0,785.0,81.916660,-0.613049,4352,237.0,222,...,0,0,0,0,0,0,0,1,1,1
1,4264440.30,4.0,10.035728,15.0,849.0,81.973876,-0.791090,4413,199.0,209,...,0,0,0,0,0,0,0,1,2,1
2,4263334.95,342.0,855.348205,16.0,870.0,81.035217,0.519791,4443,186.0,214,...,0,0,0,0,0,0,0,1,3,1
3,4262229.60,24.0,60.238933,15.0,828.0,80.054099,0.432647,4383,211.0,208,...,0,0,0,0,0,0,0,1,4,1
4,4258913.55,338.0,847.158476,15.0,892.0,77.054464,-0.923799,4473,187.0,218,...,0,0,0,0,0,0,0,1,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2062583.10,23.0,57.594866,14.0,85.0,16.059535,-0.103136,108,212.0,210,...,0,0,0,0,0,0,0,0,581008,1
581008,2059267.05,37.0,92.585707,17.0,120.0,17.977536,0.417243,90,217.0,202,...,0,0,0,0,0,0,0,0,581009,1
581009,2057056.35,35.0,87.547136,14.0,60.0,10.962117,0.902718,85,218.0,209,...,0,0,0,0,0,0,0,0,581010,1
581010,2055951.00,18.0,45.141729,13.0,95.0,14.972901,-0.202035,90,210.0,213,...,0,0,0,0,0,0,0,0,581011,1


In [8]:
df.nunique().sort_values().tail(20)


20         2
21         2
22         2
49         2
48         2
17         3
62         7
3         67
9        186
8        207
10       255
1        361
4        569
0       1978
7       5811
11      5826
2     576099
6     577988
5     581012
60    581012
dtype: int64

In [9]:
print(df.iloc[:, 9].value_counts(dropna=False))


9
231    13482
228    13474
233    13097
230    13092
229    13077
       ...  
71         1
81         1
78         1
74         1
76         1
Name: count, Length: 187, dtype: int64


In [10]:
print(df.iloc[:, 9].unique())


['222' '209' '214' '208' '218' '206' '220' '224' '205' '227' '213' '237'
 '234' '233' '226' '216' '235' '229' '219' '223' '230' '228' '225' '207'
 '232' '201' '200' '217' '202' 'a' '215' '211' '212' '231' '210' '221'
 '203' '198' '199' nan '189' '204' '197' '196' '193' '194' '195' '191'
 '236' '238' '239' '245' '248' '249' '192' '247' '186' '180' '188' '190'
 '242' '184' '178' '182' '171' '177' '173' '183' '164' '246' '240' '167'
 '241' '156' '160' '165' '243' '168' '153' '187' '250' '185' '175' '163'
 '244' '176' '157' '172' '150' '161' '170' '179' '152' '166' '169' '158'
 '162' '145' '174' '181' '254' '159' '253' '155' '251' '154' '147' '252'
 '148' '149' '151' '140' '144' '142' '130' '136' '111' '120' '146' '110'
 '143' '104' '133' '100' '134' '139' '124' '115' '99' '125' '112' '141'
 '114' '119' '122' '135' '126' '132' '128' '123' '129' '121' '138' '117'
 '118' '131' '107' '98' '113' '137' '42' '101' '95' '90' '102' '85' '116'
 '80' '109' '105' '81' '74' '91' '106' '68' '93' '108' 

### Split the DataFrame into continuous and categorical features

In [11]:
# use a threshold of 20 to split the DataFrame into continuous and categorical features
threshold = 20

df_categorical = df_noTarget.loc[:, df_noTarget.nunique(dropna=True) <= threshold]
df_continuous = df_noTarget.loc[:, df_noTarget.nunique(dropna=True) > threshold]

In [12]:
# convert numerical columns to 'category' dtype
df_categorical = df_categorical.apply(lambda x: x.astype('category') if x.dtype == 'float64' else x)
df_categorical = df_categorical.apply(lambda x: x.astype('category') if x.dtype == 'int64' else x)



In [13]:
df_continuous

,0,1,2,3,4,5,6,7,8,9,10,11,60
0,4264440.30,89.0,222.711631,11.0,785.0,81.916660,-0.613049,4352,237.0,222,115,2402.0,1
1,4264440.30,4.0,10.035728,15.0,849.0,81.973876,-0.791090,4413,199.0,209,146,2468.0,2
2,4263334.95,342.0,855.348205,16.0,870.0,81.035217,0.519791,4443,186.0,214,166,2497.0,3
3,4262229.60,24.0,60.238933,15.0,828.0,80.054099,0.432647,4383,211.0,208,130,2440.0,4
4,4258913.55,338.0,847.158476,15.0,892.0,77.054464,-0.923799,4473,187.0,218,170,2526.0,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2062583.10,23.0,57.594866,14.0,85.0,16.059535,-0.103136,108,212.0,210,133,819.0,581008
581008,2059267.05,37.0,92.585707,17.0,120.0,17.977536,0.417243,90,217.0,202,115,769.0,581009
581009,2057056.35,35.0,87.547136,14.0,60.0,10.962117,0.902718,85,218.0,209,124,832.0,581010
581010,2055951.00,18.0,45.141729,13.0,95.0,14.972901,-0.202035,90,210.0,213,138,780.0,581011


In [14]:
df_categorical.dtypes

12    category
13    category
14    category
15    category
16    category
17    category
18    category
19    category
20    category
21    category
22    category
23    category
24    category
25    category
26    category
27    category
28    category
29    category
30    category
31    category
32    category
33    category
34    category
35    category
36    category
37    category
38    category
39    category
40    category
41    category
42    category
43    category
44    category
45    category
46    category
47    category
48    category
49    category
50    category
51    category
52    category
53    category
54    category
55    category
56    category
57    category
58    category
59    category
61    category
dtype: object

### Get reports

In [15]:
latex_table = data_quality_report_latex(df_continuous)

In [16]:
print(latex_table)

\begin{table}
\caption{Data Quality Report}
\label{tab:data_quality}
\begin{tabular}{|l|r|r|r|r|r|r|r|r|r|r|}
\toprule
Feature & Count & Miss. & Card. & Min. & 1st Qrt. & Mean & Median & 3rd Qrt. & Max. & Std. Dev. \\
\midrule
0 & 581012 & 0 & 1978 & 2054845.65 & 3104928.15 & 3271134.43 & 3311628.60 & 3496222.05 & 4264440.30 & 309481.13 \\
1 & 581012 & 2943 & 361 & 0.00 & 58.00 & 155.66 & 127.00 & 260.00 & 360.00 & 111.91 \\
2 & 581012 & 0 & 576099 & 0.00 & 145.49 & 389.92 & 318.12 & 652.53 & 903.48 & 280.34 \\
3 & 581012 & 304 & 67 & 0.00 & 9.00 & 14.10 & 13.00 & 18.00 & 66.00 & 7.49 \\
4 & 581012 & 2943 & 569 & -691.00 & 108.00 & 269.42 & 218.00 & 384.00 & 1397.00 & 212.56 \\
5 & 581012 & 0 & 581012 & -173.07 & 6.99 & 46.42 & 29.91 & 68.97 & 600.95 & 58.30 \\
6 & 581012 & 2943 & 577988 & -1.00 & -0.50 & -0.00 & -0.00 & 0.50 & 1.00 & 0.58 \\
7 & 581012 & 0 & 5811 & 0.00 & 1106.00 & 8158.11 & 1997.00 & 3328.00 & 510165098.00 & 1185156.02 \\
8 & 581012 & 2943 & 207 & 0.00 & 198.00 & 212

In [17]:
# get categorical data quality report
latex_cat_table = categorical_data_quality_report_latex(df_categorical)
print(latex_cat_table)

\begin{table}
\caption{Categorical Data Quality Report}
\label{tab:categorical_quality}
\begin{tabular}{|l|r|r|r|l|r|r|l|r|r|}
\toprule
Feature & count & \%miss & Card & Mode & Mode Freq. & Mode \% & 2nd Mode & 2nd Mode Freq & 2nd Mode \% \\
\midrule
12 & 581012 & 0.00 & 2 & 0.00 & 318579 & 54.83 & 1.00 & 259490 & 44.66 \\
13 & 581012 & 0.00 & 2 & 0.00 & 551128 & 94.86 & 1 & 29884 & 5.14 \\
14 & 581012 & 0.00 & 2 & 0.00 & 327648 & 56.39 & 1 & 253364 & 43.61 \\
15 & 581012 & 0.00 & 1 & 0.00 & 578069 & 99.49 &  & 0 & 0.00 \\
16 & 581012 & 0.00 & 1 & 0.00 & 581012 & 100.00 &  & 0 & 0.00 \\
17 & 581012 & 0.00 & 3 & 0.00 & 538608 & 92.70 & 1.00 & 36589 & 6.30 \\
18 & 581012 & 0.00 & 2 & 0.00 & 291278 & 50.13 & 1 & 289734 & 49.87 \\
19 & 581012 & 0.00 & 2 & 0.00 & 574553 & 98.89 & 1.00 & 3013 & 0.52 \\
20 & 581012 & 0.00 & 2 & 0.00 & 172455 & 29.68 & 1.00 & 900 & 0.15 \\
21 & 581012 & 0.00 & 2 & 0.00 & 573487 & 98.70 & 1 & 7525 & 1.30 \\
22 & 581012 & 0.00 & 2 & 0.00 & 576189 & 99.17 & 1 & 4

---

In [18]:
target = df.iloc[:, -1]

In [19]:
np.count_nonzero(target == "?")


0

In [20]:
target = np.where(target == "?", np.nan, target)

In [21]:
target_df = pd.DataFrame(target)

In [22]:
latex_table = data_quality_report_latex(target_df)

In [23]:
latex_table

'\\begin{table}\n\\caption{Data Quality Report}\n\\label{tab:data_quality}\n\\begin{tabular}{|l|r|r|r|r|r|r|r|r|r|r|}\n\\toprule\nFeature & Count & Miss. & Card. & Min. & 1st Qrt. & Mean & Median & 3rd Qrt. & Max. & Std. Dev. \\\\\n\\midrule\n0 & 581012 & 60 & 7 & 1.00 & 1.00 & 2.05 & 2.00 & 2.00 & 7.00 & 1.40 \\\\\n\\bottomrule\n\\end{tabular}\n\\end{table}\n'

# Task 2

In [24]:
df

,0,1,2,3,4,5,6,7,8,9,...,53,54,55,56,57,58,59,60,61,62
0,4264440.30,89.0,222.711631,11.0,785.0,81.916660,-0.613049,4352,237.0,222,...,0,0,0,0,0,0,1,1,1,7.0
1,4264440.30,4.0,10.035728,15.0,849.0,81.973876,-0.791090,4413,199.0,209,...,0,0,0,0,0,0,1,2,1,7.0
2,4263334.95,342.0,855.348205,16.0,870.0,81.035217,0.519791,4443,186.0,214,...,0,0,0,0,0,0,1,3,1,7.0
3,4262229.60,24.0,60.238933,15.0,828.0,80.054099,0.432647,4383,211.0,208,...,0,0,0,0,0,0,1,4,1,7.0
4,4258913.55,338.0,847.158476,15.0,892.0,77.054464,-0.923799,4473,187.0,218,...,0,0,0,0,0,0,1,5,1,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581007,2062583.10,23.0,57.594866,14.0,85.0,16.059535,-0.103136,108,212.0,210,...,0,0,0,0,0,0,0,581008,1,3.0
581008,2059267.05,37.0,92.585707,17.0,120.0,17.977536,0.417243,90,217.0,202,...,0,0,0,0,0,0,0,581009,1,6.0
581009,2057056.35,35.0,87.547136,14.0,60.0,10.962117,0.902718,85,218.0,209,...,0,0,0,0,0,0,0,581010,1,3.0
581010,2055951.00,18.0,45.141729,13.0,95.0,14.972901,-0.202035,90,210.0,213,...,0,0,0,0,0,0,0,581011,1,3.0
